# ImportingRequiredLibraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error,r2_score
pd.options.display.max_columns = 1000
import pandas_profiling

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
macro = pd.read_excel('macro_economic.xlsx')
weather = pd.read_excel('weather.xlsx')

# Data Preprocessing

# Weather

In [3]:
weather.shape

(2922, 23)

In [4]:
# Filling the null values in WeatherEvent column with mode og=f that column
weather['WeatherEvent'] = weather['WeatherEvent'].fillna(weather['WeatherEvent'].mode().iloc[0])

In [5]:
#Finding the percentage of rows with 'T' in 'precip (mm) sum' column.
count=0
for i in weather['Precip.\xa0(mm) sum']=='T':
    if i==True:
        count=count+1
print('Total number of rows with T is', count)
print('It is',round((count/len(weather.index))*100), 'percent of the entire dataset.')

Total number of rows with T is 205
It is 7 percent of the entire dataset.


In [6]:
#As they are only 7% of data we can remove those rows.
weather['Precip.\xa0(mm) sum'] = weather['Precip.\xa0(mm) sum'].replace('T', np.nan)
weather = weather.dropna()

In [7]:
# 1826 index column has string values which is inappropriate, hence we can drop it.
weather = weather.drop([1826])

In [8]:
#Counting the percentage of rows with '-'
count=0
for i in weather['Sea Level Press.\xa0(hPa) avg']=='-':
    if i==True:
        count=count+1
print('Total number of rows with - is', count)
print('It is',round((count/len(weather.index))*100), 'percent of the entire dataset.')

Total number of rows with - is 16
It is 1 percent of the entire dataset.


In [9]:
# IT is only 1 percent of the whole data, hence we can drop these rows.
weather = weather.replace('-',np.nan)
weather = weather.dropna()

In [10]:
#Label Encoding 'WeatherEvent' column for aggregating columns by  year and month
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
weather.WeatherEvent = le.fit_transform(weather.WeatherEvent)

In [11]:
# Aggregating columns by Year and Month
weather = weather.groupby(by=['Year','Month'])[weather.columns[3:]].mean()
weather = weather.reset_index()

In [12]:
#Label encoding months with its respective number.
a = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
weather.Month = weather['Month'].map(a)

In [13]:
weather.shape

(96, 22)

# Macro

In [14]:
# Splitting year and month by '-' and storing themm as seperate columns and dropping Year-Month column
k = macro['Year-Month'].str.split(' - ',expand=True)
macro['Year'] = k[0]
macro['Month'] = k[1]
del macro['Year-Month']

In [15]:
##Mapping months with its respective number as in 'macro' dataset and label encoding months with its respective number as in 'weather' dataset
macro.Year = macro.Year.astype(int)

a = {'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
macro.Month = macro['Month'].map(a)

In [16]:
# Replacing '?' with median of AdvertisingExpenses column.
macro['AdvertisingExpenses (in Thousand Dollars)'] = macro['AdvertisingExpenses (in Thousand Dollars)'].replace('?', np.nan)
macro['AdvertisingExpenses (in Thousand Dollars)'] = macro['AdvertisingExpenses (in Thousand Dollars)'].fillna(macro['AdvertisingExpenses (in Thousand Dollars)'].median())
macro['AdvertisingExpenses (in Thousand Dollars)'] = macro['AdvertisingExpenses (in Thousand Dollars)'].astype(int)

In [17]:
# All the values in this column are same hence we can drop this column
macro = macro.drop('PartyInPower',axis=1)

In [18]:
macro.shape

(96, 18)

## Train,Test

In [19]:
# Adding 'data' columns to both train and test and filling those columns as train and test respectively.
train['data'] = 'train'
test['data'] = 'test'

In [20]:
# Counting the percentage of null valued rows in train dataset
count=0
for i in train.isnull().sum(axis=1):
    if i>0:
        count=count+1
print('Total number of rows with missing values is ', count)
print('It is',round((count/len(train.index))*100), 'percent of the entire dataset.')

Total number of rows with missing values is  12
It is 6 percent of the entire dataset.


In [21]:
# Dropping null valued rows in train as they are only 6% of total dataset.
train.dropna(inplace = True)

In [22]:
#Concating train and test datasets
train_test = pd.concat([train,test])

In [23]:
train_test.shape

(240, 5)

### Merging macro and train test

In [24]:
# Merging weather and macro datasets
final = train_test.merge(weather,on=['Year','Month'],how='inner').merge(macro,on=['Year','Month'],
                                                                        how='inner')

## Segregrating Train and test

In [25]:
train = final[final['data']=='train']
test = final[final['data']=='test']


test.drop(['data','Sales(In ThousandDollars)'],axis =1,inplace =True)
train.drop('data',axis = 1,inplace = True)

In [26]:
#Splitting full train into X(independent variables) and Y(dependent variables)
X = train.drop('Sales(In ThousandDollars)',axis =1)
y = train[['Sales(In ThousandDollars)']]

In [27]:
# One Hot encoding ProductCategory column in train
dummies_train = pd.get_dummies(X.ProductCategory,drop_first=True)
X = pd.concat([X,dummies_train],axis = 1)
X.drop('ProductCategory',axis = 1,inplace = True)

#One Hot encoding ProductCategory column in test
dummies_test = pd.get_dummies(test.ProductCategory,drop_first=True)
test = pd.concat([test,dummies_test],axis = 1)
test.drop('ProductCategory',axis = 1,inplace = True)

In [28]:
xtrain,xval,ytrain,yval = train_test_split(X,y,test_size = 0.3,random_state=1)

# Scaling 
sc = StandardScaler()

xtrain_scaled = sc.fit_transform(xtrain)
xval_scaled = sc.transform(xval)

# Modeling

# Random Forest

In [29]:
#Applying Random Forest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=300)
rf.fit(xtrain,ytrain)
ypred = rf.predict(xtrain)
rf_ypred = rf.predict(xval)
print('TRAIN')
print('RMSE: ',np.sqrt(mean_squared_error(ytrain,ypred)))
print('R2 value: ',r2_score(ytrain,ypred)*100)
print('TEST')
print('RMSE: ',np.sqrt(mean_squared_error(yval,rf_ypred)))
print('R2 value: ',r2_score(yval,rf_ypred)*100)


TRAIN
RMSE:  120.00722927069874
R2 value:  99.14353386810338
TEST
RMSE:  255.48893314310538
R2 value:  94.4165053038367


In [30]:
# rf.fit(X,y)
# csv = pd.DataFrame({'S.No.':np.arange(1,37),'Sales':rf.predict(test)})
# csv.to_csv('7_Submission_rf.csv',index=False)

# Random Forest with Hyper Parameter Tuning

In [31]:
params = {'n_estimators':[100,300,500],
          'max_features':[20,30,40],
          'min_samples_leaf':[10,15,20]
}

grid_search = GridSearchCV(rf,param_grid=params,cv = 5)
grid_search.fit(xtrain,ytrain)
grid_search.best_params_
best_rf = grid_search.best_estimator_
best_rf.fit(xtrain,ytrain)
ypred = best_rf.predict(xtrain)
best_rf_ypred = best_rf.predict(xval)
print('TRAIN')
print('RMSE: ',np.sqrt(mean_squared_error(ytrain,ypred)))
print('R2 value: ',r2_score(ytrain,ypred)*100)
print('TEST')
print('RMSE: ',np.sqrt(mean_squared_error(yval,best_rf_ypred)))
print('R2 value: ',r2_score(yval,best_rf_ypred)*100)

TRAIN
RMSE:  332.5092039709577
R2 value:  93.42489460605212
TEST
RMSE:  332.85949630756687
R2 value:  90.5227175151274


In [32]:
# best_rf.fit(X,y)
# csv = pd.DataFrame({'S.No.':np.arange(1,37),'Sales':best_rf.predict(test)})
# csv.to_csv('6_Submission_best_rf.csv',index=False)

# Ada Boost

In [33]:
from sklearn.ensemble import AdaBoostRegressor
ad = AdaBoostRegressor()

ad.fit(xtrain,ytrain)
ypred = ad.predict(xtrain)
ad_ypred = ad.predict(xval)

print('TRAIN')
print('RMSE: ',np.sqrt(mean_squared_error(ytrain,ypred)))
print('R2 value: ',r2_score(ytrain,ypred)*100)
print('TEST')
print('RMSE: ',np.sqrt(mean_squared_error(yval,ad_ypred)))
print('R2 value: ',r2_score(yval,ad_ypred)*100)


TRAIN
RMSE:  193.1961683717743
R2 value:  97.78030965027176
TEST
RMSE:  298.3425068881091
R2 value:  92.38636256551625


# AdaBoost with Hyperparameter Tuning

In [34]:
params = {'learning_rate':[1.0,1.1,1.3,1.6,0.1], 
          'loss':['linear'],
          'n_estimators':[200,400,600,700,900]
         }

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(ad,param_grid=params,cv = 5)

grid_search.fit(xtrain,ytrain)

best_ad = grid_search.best_estimator_

best_ad.fit(xtrain,ytrain)
ypred = best_ad.predict(xtrain)
best_ad_ypred = best_ad.predict(xval)

print('TRAIN')
print('RMSE: ',np.sqrt(mean_squared_error(ytrain,ypred)))
print('R2 value: ',r2_score(ytrain,ypred)*100)
print('TEST')
print('RMSE: ',np.sqrt(mean_squared_error(yval,best_ad_ypred)))
print('R2 value: ',r2_score(yval,best_ad_ypred)*100)

TRAIN
RMSE:  179.45913657781458
R2 value:  98.08474535622
TEST
RMSE:  252.1952413625487
R2 value:  94.55953904698679


In [35]:
# best_ad.fit(X,y)
# csv = pd.DataFrame({'S.No.':np.arange(1,37),'Sales':best_ad.predict(test)})
# csv.to_csv('1_Submission_best_ad.csv',index=False)

# Bagging Regressor with Hyperparameter Tuning

In [36]:
from sklearn.ensemble import BaggingRegressor
br = BaggingRegressor()

params = ({'n_estimators':[300,400,600],
    'bootstrap':[True,False],
    'bootstrap_features':[True,False]})

grid_search = GridSearchCV(br,param_grid=params,cv = 5)

grid_search.fit(xtrain,ytrain)

best_br = grid_search.best_estimator_

best_br.fit(xtrain,ytrain)
ypred = best_br.predict(xtrain)
best_br_ypred = best_br.predict(xval)

print('TRAIN')
print('RMSE: ',np.sqrt(mean_squared_error(ytrain,ypred)))
print('R2 value: ',r2_score(ytrain,ypred)*100)
print('TEST')
print('RMSE: ',np.sqrt(mean_squared_error(yval,best_br_ypred)))
print('R2 value: ',r2_score(yval,best_br_ypred)*100)

TRAIN
RMSE:  115.80009600286306
R2 value:  99.20253209198484
TEST
RMSE:  248.62133616705353
R2 value:  94.71264202779189


In [37]:
# #best_br.fit(X,y)
# csv = pd.DataFrame({'S.No.':np.arange(1,37),'Sales':best_br.predict(test)})
# csv.to_csv('3_Submission_best_br.csv',index=False)

# Gradient Boosting

In [38]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(n_estimators=300,learning_rate=0.1)

gb.fit(xtrain,ytrain)
ypred = gb.predict(xtrain)
gb_ypred = gb.predict(xval)

print('TRAIN')
print('RMSE: ',np.sqrt(mean_squared_error(ytrain,ypred)))
print('R2 value: ',r2_score(ytrain,ypred)*100)
print('TEST')
print('RMSE: ',np.sqrt(mean_squared_error(yval,gb_ypred)))
print('R2 value: ',r2_score(yval,gb_ypred)*100)

TRAIN
RMSE:  11.81417651569325
R2 value:  99.99169953759443
TEST
RMSE:  142.9811584503067
R2 value:  98.25128294687597


In [39]:
# gb.fit(X,y)
# csv = pd.DataFrame({'S.No.':np.arange(1,37),'Sales':gb.predict(test)})
# csv.to_csv('2_Submission_gb.csv',index=False)

# Gradient Boosting with Hyperparameter Tuning

In [40]:
params = {'n_estimators': [100,200,300,400], 'learning_rate': [0.1, 0.5, 1.],
           'subsample': [0.05,1.0],
           'max_features': [0.05,1.0] }

grid_search = GridSearchCV(gb,param_grid=params,cv=5)

grid_search.fit(xtrain,ytrain)

best_gb = grid_search.best_estimator_

best_gb.fit(xtrain,ytrain)
ypred = best_gb.predict(xtrain)
best_gb_ypred = best_gb.predict(xval)

print('TRAIN')
print('RMSE: ',np.sqrt(mean_squared_error(ytrain,ypred)))
print('R2 value: ',r2_score(ytrain,ypred)*100)
print('TEST')
print('RMSE: ',np.sqrt(mean_squared_error(yval,best_gb_ypred)))
print('R2 value: ',r2_score(yval,best_gb_ypred)*100)

TRAIN
RMSE:  8.346977398652168
R2 value:  99.99585662872104
TEST
RMSE:  144.61012412678
R2 value:  98.21121015752692


In [41]:
# best_gb.fit(X,y)
# csv = pd.DataFrame({'S.No.':np.arange(1,37),'Sales':best_gb.predict(test)})
# csv.to_csv('4_Submission_best_gb.csv',index=False)

# Stacking

In [44]:
#Lets stack RandomForest,AdaBoost,BaggingRegressor,GradientBooster
df = pd.DataFrame({'RF':rf.predict(X),'Ada':best_ad.predict(X),'BR':best_br.predict(X),'GB':best_gb.predict(X)})

stacked_model=RandomForestRegressor(n_estimators=300)
stacked_model.fit(df,y)
ypred = stacked_model.predict(df)
print('Train R2:', r2_score(y,ypred))

#Test
df_test = pd.DataFrame({'RF':rf.predict(test),'Ada':best_ad.predict(test),'BR':best_br.predict(test),'GB':best_gb.predict(test)})
test_ypred = stacked_model.predict(df_test)
csv = pd.DataFrame({'S.No.':np.arange(1,37),'Sales':test_ypred})
#csv.to_csv('8_Submission_stack.csv',index=False)

Train R2: 0.9988143431250104
